<a href="https://colab.research.google.com/github/Ph-Analytic/BADS7105_CRM_ANALYTICS_AND_INTELLIGENCE/blob/Ph-Analytic-patch-1/Voice_Customer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Voice Customer : Shabu shabu Restaurant

In [ ]:
!pip install --upgrade pythainlp
!pip install pyLDAvis

Requirement already up-to-date: pythainlp in /usr/local/lib/python3.7/dist-packages (2.3.1)


In [ ]:
import pandas as pd
import pythainlp
import gensim
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [ ]:
#from google.colab import files 
#uploaded = files.upload()

In [ ]:
import io
df = pd.read_csv(io.StringIO(uploaded['CustomerReviews.csv'].decode('utf-8')))

In [ ]:
df.head

<bound method NDFrame.head of     Review ID  ... Rating
0           1  ...    5.0
1           2  ...    5.0
2           3  ...    4.0
3           4  ...    4.0
4           5  ...    5.0
5           6  ...    4.0
6           7  ...    4.0
7           8  ...    4.0
8           9  ...    4.0
9          10  ...    3.0
10         11  ...    3.0
11         12  ...    3.0
12         13  ...    4.0
13         14  ...    3.0
14         15  ...    4.0
15         16  ...    4.0
16         17  ...    4.0
17         18  ...    4.0
18         19  ...    3.0
19         20  ...    5.0
20         21  ...    NaN

[21 rows x 7 columns]>

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Review ID      21 non-null     int64  
 1   Restaurant_ID  21 non-null     object 
 2   Restaurant     21 non-null     object 
 3   User           21 non-null     object 
 4   Headline       21 non-null     object 
 5   Review         21 non-null     object 
 6   Rating         20 non-null     float64
dtypes: float64(1), int64(1), object(5)
memory usage: 1.3+ KB


In [ ]:
stopwords=list(pythainlp.corpus.thai_stopwords())
removed_words = [' ', '   ' , '\n','   ', 'ร้าน', ' (' ,  ')','😆','-']
screening_words = stopwords + removed_words
def tokenize_with_space(sentence):
  merged = ' '
  words = pythainlp.word_tokenize(str(sentence), engine='newmm')
  for word in words:
     if word not in screening_words:
       merged = merged +  ',' + word
  return merged[1:]

In [ ]:
df['Review_tokenized'] = df['Review'].apply(lambda x: tokenize_with_space(x))

In [ ]:
df.tail()

,Review ID,Restaurant_ID,Restaurant,User,Headline,Review,Rating,Review_tokenized
16,17,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,ployynp,บุฟเฟ่ต์ชาบูและพิซซ่าไม่อั้นในราคา 199 บาท เน้...,หลังจากที่เคยลองสาขายูเนี่ยนมอลล์ไป รอบนี้มาที...,4.0,",หลังจากที่,ลอง,สาขา,ยู,นม,อลล์,รอบ,สาขา,เดอะ,..."
17,18,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,27a91236fe5e4559a4f097c97a480781,ร้านบุฟเฟ่ต์ ราคามิตรภาพ อยู่ชั้น4 ติดโรงหนัง ...,ร้านบุฟเฟ่ต์ที่มีโปรโมชั่นหัวละ199บาท ไม่รวมน้...,4.0,",บุฟเฟ่ต์,โปรโมชั่น,หัว,199,บาท,น้ำ,VAT,ทาน,ธร..."
18,19,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,0b81d251e4db486f9bcdba73b374ed99,ของหลากหลาย ปนๆ งงๆ นิดหน่อย,เคยรู้จักร้านนี้จากที่ union mall ไม่เคยได้ลอง...,3.0,",รู้จัก,union,mall,ลอง,กิน,จำได้,ขึ้นใจ,ชื่อ,ช..."
19,20,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,40e0e087f3914fd49a8933b5a29936ca,อร่อยมากค่ะ คุ้มค่าสมราคา บุฟเฟ่หมูผักต่างๆ รว...,อร่อยมากค่ะ คุ้มค่าสมราคา บุฟเฟ่หมูผักต่างๆ รว...,5.0,",อร่อย,คุ้มค่า,สมราคา,บุ,ฟเฟ่,หมู,ผัก,น้ำ,จบ,2..."
20,21,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,41841cb99ea243a3a8d4b006e946c586,แม้จะแปลกบ้าง แต่ก็ถือว่าอยู่ในเกณฑ์ที่ดี มีอา...,ก็ตามที่เขียนเลยครับ ว่า ถ้าจะจ่ายในราคา 199 บ...,NaN,",จ่าย,ราคา,199,บาท,จ่าย,เงินสด,ราคา,น้ำ,VAT,7,..."


In [ ]:
df.to_csv(r'Path where you want to store the exported CSV file\File Name.csv', index = False)

In [ ]:
documents = df['Review_tokenized'].to_list()
texts = [[text for text in doc.split(',')] for doc in documents]
dictionary = gensim.corpora.Dictionary(texts)

In [ ]:
print(dictionary.token2id.keys())

dict_keys(['', '1', '4', '560', 'กิน', 'ข้างนอก', 'ความต้องการ', 'คอ', 'คุณภาพ', 'คุ้ม', 'จำกัด', 'ซุป', 'ดำ', 'ดี', 'ดีมาก', 'ถาด', 'ถาม', 'น่ารัก', 'น้ำ', 'น้ำจิ้ม', 'บริการ', 'บาท', 'ประทับใจ', 'พนักงาน', 'พอกับ', 'ภาพ', 'รสชาติ', 'ราคา', 'สรุป', 'สะอาดสะอ้าน', 'สัน', 'หลากหลาย', 'อร่อย', 'อันนี้', 'อาหาร', 'เคลม', 'เซ็ท', 'เต้าเจี้ยว', 'เนื้อ', 'เป็ด', 'เผ็ด', 'แบบนี้', 'แปลก', 'แฟน', 'โดยรวม', 'โมจิ', 'ใส่ใจ', 'ไม่ต้อง', 'ไอติม', '\u200b', '           ', '21', '30', '9', 'CTW', 'Terminal', 'กา', 'การงด', 'คิดถึง', 'ชาบู', 'ญี่ปุ่น', 'ฐาน', 'ทะยอย', 'ทำงาน', 'ที่ทาง', 'นาที', 'นึกถึง', 'พนักงานบริการ', 'พระราม', 'พรีเมียม', 'พาราไดซ์', 'มาตรา', 'มีโอ', 'รายละเอียด', 'ลูกค้า', 'สมาทาน', 'สาขา', 'สู้', 'ออร์เดอร์', 'เครื่องดื่ม', 'เค้า', 'เจอ', 'เป็นกันเอง', 'เหวี่ยง', 'แจ้ง', 'โม', 'โม่', 'โอเค', '++', '2', '555', '600', ':', '::', 'กก', 'กรุบ', 'กั้น', 'ของหวาน', 'คนละ', 'คาว', 'คุ', 'คุ้มค่า', 'จาน', 'จิ้ม', 'จืด', 'ฉาก', 'ชอบ', 'ชาเขียว', 'ชิ้น', 'ซอย', 'ซอส', 'ตก', 'ตะ', 'ตัก', 

In [ ]:
gensim_corpus = [dictionary.doc2bow(text, allow_update=True) for text in texts]
word_frequencies = [[(dictionary[id], frequence) for id, frequence in couple] for couple in gensim_corpus]

In [ ]:
num_topics = 7
chunksize = 4000 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 50
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = gensim.models.LdaModel(corpus=gensim_corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

CPU times: user 367 ms, sys: 3.4 ms, total: 370 ms
Wall time: 376 ms


In [ ]:
pyLDAvis.gensim_models.prepare(model, gensim_corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.094088  0.007284       1        1  30.486085
4     -0.037317 -0.065903       2        1  27.309407
6     -0.056254  0.056073       3        1  22.037950
5     -0.002459 -0.003674       4        1  11.880546
3     -0.000404  0.003791       5        1   6.049952
2     -0.001215  0.001343       6        1   2.212557
1      0.003560  0.001086       7        1   0.023503, topic_info=       Term       Freq      Total Category  logprob  loglift
0            29.000000  29.000000  Default  30.0000  30.0000
118     ทาน  23.000000  23.000000  Default  29.0000  29.0000
18      น้ำ  23.000000  23.000000  Default  28.0000  28.0000
32    อร่อย  19.000000  19.000000  Default  27.0000  27.0000
59     ชาบู  19.000000  19.000000  Default  26.0000  26.0000
..      ...        ...        ...      ...      ...      ...
20   บริการ   0.001725   7.445251   Topic7  -5.2899  -0.0140
4       กิน   0.003181  20.247313   Topic7  -4.6783  -0.4029
279      ชิ   0.001714   7.705861   Topic7  -5.2968  -0.0554
76     สาขา   0.001954  10.803139   Topic7  -5.1653  -0.2617
276    กุ้ง   0.001793  11.308784   Topic7  -5.2518  -0.3939

[456 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
0         1  0.206569         
0         2  0.344281         
0         3  0.275425         
0         4  0.103284         
0         5  0.068856         
...     ...       ...      ...
48        2  0.373150    ไอติม
48        3  0.248767    ไอติม
48        4  0.124383    ไอติม
273       3  0.882627  ไอศครีม
49        2  0.942344        ​

[365 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 5, 7, 6, 4, 3, 2])

In [ ]:
model.show_topic(1)

[('', 0.018919742),
 ('ทาน', 0.015720233),
 ('น้ำ', 0.013457938),
 ('ชาบู', 0.013235017),
 ('ดี', 0.012812861),
 ('อร่อย', 0.012809304),
 ('เลือก', 0.011719753),
 ('ราคา', 0.011436539),
 ('เนื้อ', 0.011144186),
 ('อาหาร', 0.010635127)]

In [ ]:
model.show_topic(2)

[('', 0.02025396),
 ('ชาบู', 0.015203052),
 ('ดี', 0.014831236),
 ('อร่อย', 0.014820485),
 ('ทาน', 0.014003113),
 ('เลือก', 0.013856639),
 ('เนื้อ', 0.013343994),
 ('น้ำ', 0.01198389),
 ('ชอบ', 0.0109001845),
 ('คุ้ม', 0.010351029)]

In [ ]:
model.show_topic(3)

[('ทาน', 0.023232548),
 ('', 0.019871585),
 ('อร่อย', 0.018181067),
 (':', 0.016892878),
 ('เลือก', 0.014500294),
 ('รสชาติ', 0.013412185),
 ('ชาบู', 0.012762634),
 ('ดี', 0.012449216),
 ('ราคา', 0.011418206),
 ('อาหาร', 0.0108218435)]

In [ ]:
df['topics'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][0])
df['score'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][1])

In [ ]:
df[['Restaurant', 'Review', 'topics']]

,Restaurant,Review,topics
0,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...,4
1,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,นึกถึงชาบูญี่ปุ่นยังไงก็ต้อง คิดถึงโมโม่ พาราไ...,5
2,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,มาทานช่วงนี้ สามารถนั่งโต๊ะเดียวกัน หม้อเดียวก...,6
3,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ถ้านึกถึงชาบูที่มีเนื้อเน้นๆ ในราคาไม่โหดจนเกิ...,6
4,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,เดินมาหน้าร้านแล้วได้กลิ่นชาบูหอมมาก ๆ ประกอบ...,0
5,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ร้านบุฟเฟ่ ชาบูแนวญี่ปุ่น สายเนื้อหมู เนื้อวัว...,2
6,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,Number 20 : โมโม – พาราไดส์ (สาขาเดอะมอลบางกะป...,4
7,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ร้านชาบูเฟรนไชส์รสชาติดีมากคุ้มค่าเหมาะสมกับรา...,6
8,Shabushi (ชาบูชิ) เดอะมอลล์บางกะปิ ชั้น G,มา านที่ขาบูชิต้องมาตอนหิว ไม่งั้นจะไม่คุ้มนะค...,5
9,Shabushi (ชาบูชิ) เดอะมอลล์บางกะปิ ชั้น G,ใครชอบกุ้งทอดเทมปุระ แค่กุ้งเทมปุระก็คุ้มแล้ว ...,4
